In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from IPython.display import clear_output
import datetime

In [6]:
# 요청 함수
def getSource(site) :
    # 해더 정보 셋팅
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    # 요청한다.
    response = requests.get(site, headers=header_info)
    return response.text

In [113]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source):
    
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # 영화 전체를 가져온다.
    li_list = soup.select('#content > div.article > div > div.lst_wrap > ul > li')
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        '제목' : [],
        '평점' : [],
        '개요' : [],
        '감독' : [],
        '출연' : [],
        '상영시간' : [],
        '개봉날짜' : [],
        '영화코드' : []
    }
    
    for li_tag in li_list :
        # print(li_tag)
        # print('---------------------------')

        # 제목
        a1 = li_tag.select_one('dl > dt > a')
        data1 = getText(a1)
        
        # 평점
        a2 = li_tag.select_one('dl > dd.star > dl > dd > div > a > span.num')
        data2 = getText(a2)
        
        # 개요
        a3 = li_tag.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(2) > span.link_txt')
        data3 = getText(a3)
        if pd.notna(data3) :
            data3 = data3.replace('\r', '')
            data3 = data3.replace('\n', '')
            data3 = data3.replace('\t', '')
            data3 = data3.replace(' ', '')
        
        # 감독
        a4 = li_tag.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(4) > span')
        data4 = getText(a4)
        if pd.notna(data4) :
            data4 = data4.replace('\r', '')
            data4 = data4.replace('\n', '')
            data4 = data4.replace('\t', '')
            data4 = data4.replace(' ', '')
        
        # 출연
        a5 = li_tag.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(6) > span')
        data5 = getText(a5)
        if pd.notna(data5) :
            data5 = data5.replace('\r', '')
            data5 = data5.replace('\n', '')
            data5 = data5.replace('\t', '')
            data5 = data5.replace(' ', '')
            
        # 개봉일, 상영시간
        a6 = li_tag.select_one(' dl > dd:nth-child(3) > dl > dd:nth-child(2)')
        data6 = getText(a6)
        # |를 기준으로 잘라낸다.
        a7 = data6.split('|')
        open_date = np.nan
        play_time = np.nan
        for a8 in a7 :
            a8 = a8.strip()
            # 분으로 끝난다면
            if a8.endswith('분') :
                play_time = a8.replace('분', '')
                play_time = play_time.strip()
            # 개봉으로 끝난다면
            if a8.endswith('개봉'):
                open_date = a8.replace('개봉', '')
                open_date = open_date.strip()
        # 영화코드
        a9 = li_tag.select_one('dl > dt > a')
        a10 = a9.attrs['href']
        # =을 기준으로 잘라낸다.
        a11 = a10.split('=')
        code = a11[-1]
        # print(code)
        
        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        # print(data5)
        # print(play_time)
        # print(open_date)
        # print('---------------------------')
        
        # 담는다.
        data_dict['제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['개요'].append(data3)
        data_dict['감독'].append(data4)
        data_dict['출연'].append(data5)
        data_dict['상영시간'].append(play_time)
        data_dict['개봉날짜'].append(open_date)
        data_dict['영화코드'].append(code)
        
    df1 = pd.DataFrame(data_dict)
    df1.to_csv('./04_data.csv', encoding='utf-8-sig', index=False)

In [114]:
# 전달된 태그로 부터 문자열 데이터를 추출해 반환하는 함수
def getText(tag):
    # 태그가 없다면
    if tag == None :
        return np.nan
    # 문자열을 가져온다.
    data = tag.text.strip()
    # 길이가 0 이라면...
    if len(data) == 0 :
        return np.nan
    return data

In [115]:
# 페이지의 주소
site = 'https://movie.naver.com/movie/running/current.naver#'

source = getSource(site)

getData(source)

print('수집완료')

수집완료


In [2]:
help(datetime)

Help on module datetime:

NAME
    datetime - Fast implementation of the datetime type.

MODULE REFERENCE
    https://docs.python.org/3.9/library/datetime
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

CLASSES
    builtins.object
        date
            datetime
        time
        timedelta
        tzinfo
            timezone
    
    class date(builtins.object)
     |  date(year, month, day) --> date object
     |  
     |  Methods defined here:
     |  
     |  __add__(self, value, /)
     |      Return self+value.
     |  
     |  __eq__(self, value, /)
     |      Return self==value.
     |  
     |  __format__(...)
     |      Formats self with strftime.
     |  
     |  __ge__(self, value, /)
     |    